In [24]:
runGame()

GAME START
PC: [1, 3, 12, 11]
CC: [7, 8, 9, 6]
These are your cards: [1, 3, 12, 11]
rounds: 1
P1: [1, 0, 2, 0]
P1: [1, 0, 2, 0]
C1: [1, 0, 6, -1]
C1: [1, 0, 6, -1]
rounds: 2
P2: [1, 0, 12, 0]
P2: [1, 0, 12, 0]
C2: [2, 0, 6, -1]
C2: [2, 0, 6, -1]
rounds: 3
P3: bluff
P3: bluff
PC: [1, 3, 12, 11]  CC: [7, 8, 9, 6]
GAME END



In [1]:
import math
#import numpy
import autograd.numpy as np
import random
from autograd import elementwise_grad
from scipy.special import comb
import itertools

In [2]:
import logging
import sys

In [1]:
logger = logging.getLogger()

logger.setLevel(logging.INFO)
fileOut= logging.FileHandler("test.txt")
fileOut.setLevel(logging.INFO)
#debugInfo = logging.StreamHandler(sys.stdout)
#debugInfo.setLevel(logging.DEBUG)
#logger.addHandler(debugInfo)
logger.addHandler(fileOut)


NameError: name 'logging' is not defined

In [25]:
logger.handlers


[<StreamHandler stdout (DEBUG)>,
 <FileHandler d:\Users\willi\Desktop\spubunut\test.txt (INFO)>]

In [5]:
state = np.array([1.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
numcards = 8
ownCards = np.array([0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
factor = 1
bluffEpsilon = 0.1  #remember to implement
callsList = np.array([[-1, -1, -1, -1, -1],
                  [ 0, -1, -1, -1, -1],
                  [ 1, -1,  2, -1, -1],
                  [ 3, -1,  4, -1, -1],
                  [ 5, -1, -1,  6, -1]])

def findCall(num):
    for i in range(callsList.size):
        for j in range(callsList[0].size):
            if callsList[i][j] == num:
                return [i, j]



In [6]:
def oneCardProbabilities(state):
    
    total = np.sum(state)
    oneCardProb = (state+1)/max((total+len(state)),1)
    return oneCardProb

In [7]:
def twotypeCall(state, numcards, countOne, countTwo, ownCards, typeOne, typeTwo):
    """ Output is a number representing probability of individual call
    state vector of size 13
    numcards, countOne, countTwo, typeOne, typeTwo numbers
    ownCards vector of size 13
    
    Gets specific call probability from general matrix
    """
    return twotype(state, numcards, countOne, countTwo, ownCards)[typeOne][typeTwo]

def twotype(state, numcards, countOne, countTwo, ownCards):
    """ Output is a 13x13 matrix 
    state vector of size 13
    numCards, countOne, countTwo numbers
    ownCards vector of size 13
    
    Converts state to 2x13x13 matrix
    """

    prob = oneCardProbabilities(state)
    #probPairs = np.array(np.meshgrid(prob,prob)) # so that it has shape (2,13,13)
    probPairs = np.transpose(np.reshape(np.array(list(map(np.array,itertools.product(prob,prob)))),(state.size,state.size,2)),(2,0,1))
    
             #print(probPairs.shape)
#     probabilities = np.zeroes([len(state), len(state)])
#     for i in range(len(state)):
#         for j in range(len(state)):
#             twopairprob(probPairs[i, j], numcards)
    
    
    #probPairs[0] rows, probPairs[1] columns
    return twotypeprob(probPairs, numcards, countOne, countTwo, ownCards)

def twotypeprob(p, numcards, countCardOne, countCardTwo, ownCards):
    """ Output is a 13x13 matrix. 
    p.shape=2x13x13 >state made into grid representing pairs
    numcards, countCardOne, countCardTwo numbers
    ownCards vector of size 13
    
    Returns matrix representing probabilities of calls
    """
    prob = 0
    for i in range(countCardOne, 5):
        for j in range(countCardTwo, 5): #one of the reshapes is unnecessary
            prob += (comb(numcards, i-ownCards).reshape(ownCards.shape[0], 1) * (p[0])**(i-ownCards).reshape(ownCards.shape[0],1)*\
                     comb(numcards-i+ownCards, j-ownCards).reshape(1,ownCards.shape[0]) * (p[1])**(j-ownCards).reshape(1, ownCards.shape[0])*\
                     (1-(p[0]+p[1]))**(numcards-(i+j-ownCards-ownCards.T)))
                    #col*(identicalcols**col)
                    # row*(identicalrows**row)
                    # 13x13 matrix
    return prob
            

def onetypeCall(state, numcards, count, ownCards, cardType):
    return onetype(state, numcards, count, ownCards)[cardType]
    
def onetype(state, numcards, count, ownCards):
    prob = oneCardProbabilities(state)
    return onetypeprob(prob, numcards, count, ownCards)
    
def onetypeprob(p, numcards, countCard, ownCards):
    prob = 0
    for i in range(countCard, 5):
        prob = prob + comb(numcards, i-ownCards)* p**(i-ownCards) * (1-p)**(numcards-i+ownCards)
    return prob

In [8]:
pgrad = elementwise_grad(twotypeCall)
pograd = elementwise_grad(onetypeCall)

In [9]:
def updateParameters(call, state, numcards, ownCards):
    #for i in range(10):
    if(call[1] == 0):
       grad = pograd(state, numcards, call[0],  ownCards, call[2])
    else:
       grad= pgrad(state, numcards, call[0], call[1], ownCards, call[2], call[3])
        #state = state + grad/grad.max()/10
    logger.debug(np.exp(grad))
    logger.debug(grad)
    
    return state*np.exp((grad))

In [10]:
def makeCall(call, state, numcards, ownCards, calls):  #not quite
    allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]    
    ind = calls[call[0]][call[1]]
    bestCall = [-1, -1, -1, -1, 0]
    for i in range(ind+1, len(allCallProb)):  #call strength
        for j in range(allCallProb[i].shape[0]):  #level within call
            if len(allCallProb[i].shape) > 1:
                for k in range(allCallProb[i][j].shape[0]):
                    prob = allCallProb[i][j][k]
                    if((prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)) and k!=j):
                        
                        newcall = findCall(i)
                        bestCall = [newcall[0], newcall[1], j, k, prob]
                    
            else:
                
                prob = allCallProb[i][j]
                if(prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)):
                    newcall = findCall(i)
                    bestCall = [newcall[0], newcall[1], j, -1, prob]
                    
    currLevelProb = allCallProb[ind]
    
    for i in range(call[2], currLevelProb.shape[0]):
        if len(currLevelProb.shape) > 1:
            for j in range(currLevelProb.shape[1]):
                if(i != call[2] or j > call[3]):
                    prob = currLevelProb[i][j]
                    if((prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)) and i != j):
                        
                        newcall = findCall(ind)
                        
                        bestCall = [newcall[0], newcall[1], i, j, prob]
        else:
            if(i != call[2]):
                prob = currLevelProb[i]
                if(prob > bestCall[4] or (prob == bestCall[4] and random.random()>0.5)):
                    newcall = findCall(ind)
                    bestCall = [newcall[0], newcall[1], i, -1, prob]
    return bestCall
    

In [11]:
def makeDecision(call, state, numcards, ownCards, calls, numrounds):
    #somewhat unnecessary
    allCallProb = [onetype(state, numcards, 1, ownCards), onetype(state, numcards, 2, ownCards), twotype(state, numcards, 2, 2, ownCards), onetype(state, numcards, 3, ownCards), twotype(state, numcards, 3, 2, ownCards), onetype(state, numcards, 4, ownCards), twotype(state, numcards, 4, 3, ownCards)]

    ownCall = makeCall(call, state, numcards, ownCards, calls)
    oppCallProb = 0
    if(call[1] == 0):
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]]
    else:
        
        oppCallProb = allCallProb[calls[call[0]][call[1]]][call[2]][call[3]]
    logger.debug("probability: " + str(oppCallProb))
    oppCallProb = (1-oppCallProb)*(1-0.80**(numrounds//2+1))
    logger.debug(oppCallProb)
    if ownCall[4] >= oppCallProb:
        return 1  #call
    else:
        return 0  #bluff

In [12]:
def runGame():
    logToFile("GAME START")
    bluffCalled = 0

    numPlayerCards = 4
    numCpuCards = 4
    deckSize = 13
    factor = 1
    
    
    deck = [i for i in range(deckSize) for j in range(4)]
    random.shuffle(deck)
    playerCards=deck[0:4]
    cpuCards = deck[4:8]
    #playerCards = [12, 0, 6, 3]
    #cpuCards = [9, 0, 10, 9]
    
    logToFile("AC: "+str(playerCards))
    logToFile("BC: "+str(cpuCards))
    aState = np.array([1.0 for i in range(deckSize)])
    bState = np.array([1.0 for i in range(deckSize)])
    aCards = np.array([0 for i in range(deckSize)])
    bCards = np.array([0 for i in range(deckSize)])
    
    for i in cpuCards:
        ownCards[i] = ownCards[i]+1
        
    print('These are your cards:', playerCards)
    #print(cpuCards)
    previousProbs = [0]
    numrounds= -1
    
    while not bluffCalled:
        numrounds = numrounds+1
        
        print("rounds: " + str(numrounds+1))
        
        
        
        userIn = input('Make your decision:')
        
        if(userIn == "bluff"):
            logToFile("P" + str(numrounds+1)+ ": " + userIn)
            print("P" + str(numrounds+1)+ ": " + userIn)
            
            bluffCalled = 1
            logging.debug(previousProbs)
            break
        else:
            userIn=userIn.split()
            call= np.array([int(userIn[0]), int(userIn[1]), int(userIn[2]), int(userIn[3])])
            logToFile("P" + str(numrounds+1)+ ": " + str([int(userIn[0]), int(userIn[1]), int(userIn[2]), int(userIn[3])]))
            print("P" + str(numrounds+1)+ ": " + str([int(userIn[0]), int(userIn[1]), int(userIn[2]), int(userIn[3])]))
      
            state= updateParameters(call, state, numPlayerCards, ownCards)
           
            
            
            logger.debug(state)
            decision = makeDecision(call, state, numPlayerCards, ownCards, callsList, numrounds)
            if not decision:
                
                logToFile("C" + str(numrounds+1)+ ": bluff")
                
                print("Bluff")
                logging.debug(makeCall(call, state, numPlayerCards, ownCards, callsList))
                bluffCalled = 1
                
            else:
                k=makeCall(call, state, numPlayerCards, ownCards, callsList)[0:4]
                logToFile("C" + str(numrounds+1)+ ": "+ str(k))

                print("C" + str(numrounds+1)+ ": "+ str(k))
                previousProbs.append(makeCall(call, state, numPlayerCards, ownCards, callsList)[4])
                
            
        
            
            
    print("PC: " + str(playerCards)+"  CC: "+ str(cpuCards))
    logToFile("GAME END")
    logToFile("")

In [26]:
runGameTwo()

These are your cards: [6, 12, 0, 1]
[11, 7, 4, 7]


In [63]:
def func(x):
    logger.info("etc")
    print(x**2)
    

In [46]:
func(4)

etc
16


In [13]:
def logToFile(x):
    logger.info(x)
    

In [59]:
logToFile("ads")

ads
